# Level‑2 Adaptive Self‑Regulation (Essential, Validity‑Adjusted) — Multi‑Run

**Level‑2 claim under test**
> The system preserves functional stability under perturbation through **internally mediated parameter/state adjustment**, rather than purely reactive stimulus control.

### What was adjusted vs the prior notebook
1) **Recovery time metric fixed**: we now measure recovery **after the first post‑switch violation**, not simply “first in‑band sample after switch.”
2) **Reactive baseline strengthened (and made comparable)**: we **tune** a proportional gain `k` on a separate tuning set, then evaluate on a fresh test set.

### Minimal falsifiability structure
- **Reactive (tuned)** baseline (no internal update)
- **Adaptive (Level‑2)** candidate (internal estimate updated from prediction error)
- **Frozen update** ablation (same architecture, updates disabled)
- **Random update** ablation (internal drift without error‑link)

Multi‑run evaluation aggregates across many seeds × episodes with bootstrap confidence intervals.

Conscious Machines — Research (Feb 2026)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)


## 1) Environment (minimal but discriminative)

We regulate a single variable $x_t$ inside a tight viability band $[x_{min}, x_{max}]$.

Hidden dynamics parameter $\theta_t$ switches at time $t_p$ (unknown to the agent).
After the switch, a constant drift term $d$ is applied (also hidden). This creates a genuine destabilizing perturbation.

\[
x_{t+1} = x_t + \theta_t u_t - \lambda (x_t - x_{env}) + d_t
\]
- Before switch: $d_t = 0$
- After switch: $d_t = d$


In [ ]:
from dataclasses import dataclass

@dataclass
class EnvCfg:
    horizon: int = 90
    x_env: float = 0.0
    leak: float = 0.05
    u_max: float = 1.0
    x_min: float = -0.6
    x_max: float =  0.6
    x_target: float = 0.0

@dataclass
class Perturb:
    theta_pre: float
    theta_post: float
    drift_post: float
    switch_t: int

class HomeoEnv:
    def __init__(self, cfg: EnvCfg, perturb: Perturb, x0: float):
        self.cfg = cfg
        self.p = perturb
        self.x = float(x0)
        self.t = 0

    def theta(self):
        return self.p.theta_pre if self.t < self.p.switch_t else self.p.theta_post

    def drift(self):
        return 0.0 if self.t < self.p.switch_t else self.p.drift_post

    def observe(self):
        return float(self.x)  # θ and drift are hidden

    def step(self, u: float):
        u = float(np.clip(u, -self.cfg.u_max, self.cfg.u_max))
        th = self.theta()
        d = self.drift()
        x_next = self.x + th * u - self.cfg.leak * (self.x - self.cfg.x_env) + d
        self.x = float(x_next)
        self.t += 1
        return self.observe()

def sample_perturb(rng: np.random.Generator, horizon: int) -> Perturb:
    theta_pre = float(rng.uniform(0.6, 1.4))
    # Sometimes sign flip (hard), otherwise magnitude shift
    if rng.random() < 0.35:
        theta_post = -float(rng.uniform(0.4, 1.2))
    else:
        theta_post = float(rng.uniform(0.2, 2.0))
    drift_post = float(rng.uniform(0.04, 0.12)) * (1 if rng.random() < 0.5 else -1)
    switch_t = int(rng.integers(horizon//3, 2*horizon//3))
    return Perturb(theta_pre=theta_pre, theta_post=theta_post, drift_post=drift_post, switch_t=switch_t)


## 2) Controllers (essential set)

- **ReactiveP**: proportional controller $u_t = k(x^*-x_t)$
- **AdaptiveTheta**: maintains an internal estimate $\hat{\theta}_t$ and updates it from prediction error
- **FrozenAdaptive**: same as adaptive but updates disabled
- **RandomUpdateAdaptive**: same as adaptive but random drift instead of error‑driven update


In [ ]:
class ReactiveP:
    def __init__(self, cfg: EnvCfg, k: float = 1.2):
        self.cfg = cfg
        self.k = float(k)

    def reset(self, rng: np.random.Generator):
        pass

    def act(self, x: float) -> float:
        u = self.k * (self.cfg.x_target - x)
        return float(np.clip(u, -self.cfg.u_max, self.cfg.u_max))

class AdaptiveTheta:
    def __init__(self, cfg: EnvCfg, theta_init: float = 1.0, alpha: float = 0.35, eps: float = 1e-6):
        self.cfg = cfg
        self.theta_init = float(theta_init)
        self.alpha = float(alpha)
        self.eps = float(eps)

    def reset(self, rng: np.random.Generator):
        self.theta_hat = float(self.theta_init)
        self.rng = rng

    def predict(self, x: float, u: float) -> float:
        return float(x + self.theta_hat * u - self.cfg.leak * (x - self.cfg.x_env))

    def update(self, x: float, u: float, x_next: float):
        x_hat_next = self.predict(x, u)
        e = x_next - x_hat_next
        self.theta_hat = float(self.theta_hat + self.alpha * (e * u) / (u*u + self.eps))
        self.theta_hat = float(np.clip(self.theta_hat, -3.0, 3.0))

    def act(self, x: float) -> float:
        desired = (self.cfg.x_target - x) + self.cfg.leak * (x - self.cfg.x_env)
        denom = self.theta_hat
        if abs(denom) < 0.05:
            denom = 0.05 * np.sign(denom if denom != 0 else 1.0)
        u = desired / denom
        return float(np.clip(u, -self.cfg.u_max, self.cfg.u_max))

class FrozenAdaptive(AdaptiveTheta):
    def update(self, x: float, u: float, x_next: float):
        return

class RandomUpdateAdaptive(AdaptiveTheta):
    def __init__(self, cfg: EnvCfg, theta_init: float = 1.0, noise: float = 0.12):
        super().__init__(cfg, theta_init=theta_init, alpha=0.0)
        self.noise = float(noise)

    def update(self, x: float, u: float, x_next: float):
        self.theta_hat = float(self.theta_hat + self.noise * self.rng.normal())
        self.theta_hat = float(np.clip(self.theta_hat, -3.0, 3.0))


## 3) Metrics (fixed recovery time)

Primary metrics:
- **Violation rate (overall)**
- **Violation rate (post‑switch)**
- **Integrated deviation** (distance outside band)
- **Recovery time after first post‑switch violation** (fixed)

### Recovery time definition
Let $t_v$ be the first time *after the switch* that $x_t$ leaves the band.
Recovery time is the number of steps after $t_v$ until $x_t$ returns inside the band and stays inside for `N_stable` consecutive steps.
If there is no post‑switch violation, recovery time is 0.


In [ ]:
def in_band(x: float, lo: float, hi: float) -> bool:
    return (lo <= x <= hi)

def dist_to_band(x: float, lo: float, hi: float) -> float:
    if x < lo: return lo - x
    if x > hi: return x - hi
    return 0.0

def recovery_time_after_first_violation(xs: np.ndarray, switch_t: int, lo: float, hi: float, N_stable: int = 5) -> int:
    t_v = None
    for t in range(switch_t, len(xs)):
        if not in_band(xs[t], lo, hi):
            t_v = t
            break
    if t_v is None:
        return 0
    for t in range(t_v + 1, len(xs) - N_stable + 1):
        ok = True
        for k in range(N_stable):
            if not in_band(xs[t + k], lo, hi):
                ok = False
                break
        if ok:
            return t - t_v
    return len(xs) - 1 - t_v

def run_episode(cfg: EnvCfg, perturb: Perturb, agent, rng: np.random.Generator, x0: float):
    env = HomeoEnv(cfg, perturb, x0=x0)
    agent.reset(rng)

    xs = np.zeros(cfg.horizon)
    theta_hat = np.full(cfg.horizon, np.nan)

    for t in range(cfg.horizon):
        x = env.observe()
        u = agent.act(x)
        x_next = env.step(u)
        if hasattr(agent, 'update'):
            agent.update(x, u, x_next)
        xs[t] = x_next
        theta_hat[t] = getattr(agent, 'theta_hat', np.nan)

    lo, hi = cfg.x_min, cfg.x_max
    viol = ((xs < lo) | (xs > hi)).astype(float)
    viol_rate = float(viol.mean())
    viol_rate_post = float(viol[perturb.switch_t:].mean())
    int_dev = float(np.sum([dist_to_band(x, lo, hi) for x in xs]))
    rec_fixed = float(recovery_time_after_first_violation(xs, perturb.switch_t, lo, hi, N_stable=5))

    return {
        'xs': xs,
        'theta_hat': theta_hat,
        'violation_rate': viol_rate,
        'violation_rate_post': viol_rate_post,
        'integrated_deviation': int_dev,
        'recovery_time_fixed': rec_fixed,
    }


## 4) Tune a strong reactive baseline (separate tuning set)

We tune `k` on a separate tuning set (different seed) to avoid an underpowered reactive baseline.
Then we evaluate on a fresh test set.


In [ ]:
def tune_reactive_k(cfg: EnvCfg, k_grid: np.ndarray, n_episodes: int = 500, seed: int = 10001):
    rng = np.random.default_rng(seed)
    scores = []
    for k in k_grid:
        agent = ReactiveP(cfg, k=float(k))
        vals = []
        for _ in range(n_episodes):
            perturb = sample_perturb(rng, cfg.horizon)
            x0 = float(rng.uniform(-0.5, 0.5))
            out = run_episode(cfg, perturb, agent, rng, x0)
            vals.append(out['violation_rate_post'])
        scores.append((float(k), float(np.mean(vals))))
    scores = sorted(scores, key=lambda x: x[1])
    best_k, best_score = scores[0]
    return best_k, best_score, pd.DataFrame(scores, columns=['k','mean_post_switch_violation'])

cfg = EnvCfg()
k_grid = np.linspace(0.4, 2.5, 18)
best_k, best_score, tune_df = tune_reactive_k(cfg, k_grid, n_episodes=500, seed=10001)
best_k, best_score


In [ ]:
tune_df.sort_values('mean_post_switch_violation').head(8)


## 5) Multi‑run evaluation (test set)


In [ ]:
def bootstrap_ci(values: np.ndarray, rng: np.random.Generator, n_boot: int = 2000, alpha: float = 0.05):
    values = np.asarray(values, dtype=float)
    mean = float(values.mean())
    if len(values) < 2:
        return mean, mean, mean
    n = len(values)
    boots = np.empty(n_boot)
    for i in range(n_boot):
        idx = rng.integers(0, n, size=n)
        boots[i] = values[idx].mean()
    lo = float(np.quantile(boots, alpha/2))
    hi = float(np.quantile(boots, 1 - alpha/2))
    return mean, lo, hi

def run_multirun(cfg: EnvCfg, reactive_k: float, n_seeds: int = 25, episodes_per_seed: int = 90, seed0: int = 42000):
    agents = {
        'Reactive (tuned)': ReactiveP(cfg, k=float(reactive_k)),
        'Adaptive (Level‑2)': AdaptiveTheta(cfg, theta_init=1.0, alpha=0.35),
        'Frozen Update (Ablation)': FrozenAdaptive(cfg, theta_init=1.0, alpha=0.35),
        'Random Update (Ablation)': RandomUpdateAdaptive(cfg, theta_init=1.0, noise=0.12),
    }

    rows=[]
    examples={}
    for s in range(n_seeds):
        rng = np.random.default_rng(seed0 + s)
        for ep in range(episodes_per_seed):
            perturb = sample_perturb(rng, cfg.horizon)
            x0 = float(rng.uniform(-0.5, 0.5))
            for name, agent in agents.items():
                out = run_episode(cfg, perturb, agent, rng, x0)
                rows.append({
                    'seed': seed0 + s,
                    'episode': ep,
                    'agent': name,
                    'violation_rate': out['violation_rate'],
                    'violation_rate_post': out['violation_rate_post'],
                    'integrated_deviation': out['integrated_deviation'],
                    'recovery_time_fixed': out['recovery_time_fixed'],
                    'theta_pre': perturb.theta_pre,
                    'theta_post': perturb.theta_post,
                    'drift_post': perturb.drift_post,
                    'switch_t': perturb.switch_t,
                })
                if name not in examples and s == 0 and ep == 0:
                    examples[name] = {**out, **perturb.__dict__}
    return pd.DataFrame(rows), examples

df, examples = run_multirun(cfg, reactive_k=best_k, n_seeds=25, episodes_per_seed=90, seed0=42000)
df.head()


## 6) Aggregate results + confidence intervals


In [ ]:
rng_ci = np.random.default_rng(777)
metrics = ['violation_rate_post','integrated_deviation','recovery_time_fixed','violation_rate']

rows=[]
for agent, g in df.groupby('agent'):
    r={'agent': agent}
    for m in metrics:
        mean, lo, hi = bootstrap_ci(g[m].to_numpy(), rng_ci, n_boot=2500)
        r[f'{m}_mean']=mean
        r[f'{m}_lo']=lo
        r[f'{m}_hi']=hi
    rows.append(r)

summary = pd.DataFrame(rows).sort_values('agent')
summary


## 7) Plots


In [ ]:
def bar_ci(summary_df: pd.DataFrame, metric: str, title: str, ylabel: str):
    agents = summary_df['agent'].tolist()
    means = summary_df[f'{metric}_mean'].to_numpy()
    lo = summary_df[f'{metric}_lo'].to_numpy()
    hi = summary_df[f'{metric}_hi'].to_numpy()
    yerr = np.vstack([means - lo, hi - means])
    plt.figure(figsize=(10,4))
    plt.bar(np.arange(len(agents)), means, yerr=yerr, capsize=6)
    plt.xticks(np.arange(len(agents)), agents, rotation=25, ha='right')
    plt.title(title)
    plt.ylabel(ylabel)
    plt.tight_layout()
    plt.show()

bar_ci(summary, 'violation_rate_post', 'Post‑switch Violation Rate (lower is better)', 'Fraction outside band (post‑switch)')
bar_ci(summary, 'recovery_time_fixed', 'Recovery Time After First Violation (lower is better)', 'Steps to stable in‑band (N=5)')
bar_ci(summary, 'integrated_deviation', 'Integrated Deviation From Band (lower is better)', 'Sum distance outside band')


In [ ]:
# Example trajectories and θ̂(t)
plt.figure(figsize=(10,5))
for name, ex in examples.items():
    plt.plot(ex['xs'], label=name)
plt.axhline(cfg.x_min, linestyle='--')
plt.axhline(cfg.x_max, linestyle='--')
plt.axvline(list(examples.values())[0]['switch_t'], linestyle=':')
plt.title('Example x(t) trajectories (band dashed, switch dotted)')
plt.xlabel('t'); plt.ylabel('x'); plt.legend(); plt.tight_layout(); plt.show()

plt.figure(figsize=(10,4))
for name, ex in examples.items():
    if 'Adaptive' in name or 'Frozen' in name or 'Random' in name:
        plt.plot(ex['theta_hat'], label=name)
plt.axvline(list(examples.values())[0]['switch_t'], linestyle=':')
plt.title('Internal estimate θ̂(t) (adaptation signature)')
plt.xlabel('t'); plt.ylabel('θ̂'); plt.legend(); plt.tight_layout(); plt.show()


## 8) Level‑2 falsifiability checklist

Support for Level‑2 in this notebook corresponds to the following pattern:

1) **Adaptive (Level‑2)** outperforms **Reactive (tuned)** on post‑switch stability.

2) **Frozen Update** performs **worse than Adaptive** (updating is causally important for the observed improvement).

3) **Random Update** performs poorly (rules out “any internal drift”).
